In [3]:
import random
from deap import base, creator, tools

In [4]:
#평가 함수
def eval_func(individual) : 
    target_sum = 45
    return len(individual) - abs(sum(individual) - target_sum),


def create_toolbox(num_bits) : #각 변수나 연산자가 등록된 toolbox를 만들어 반환하는 함수
    creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #적합도를 최적화한다는 목표
    creator.create("Individual", list, fitness=creator.FitnessMax) #개체 생성 방법 설정
    
    #속성 초기화
    toolbox = base.Toolbox()
    #0,1 중 하나를 임의로 생성하는 난수 발생기
    toolbox.register("attr_bool", random.randint, 0, 1)
    #individual 함수 등록
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, num_bits)
    #개체 리스트 형태로 집단 정의하기
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    #평가 함수 등록
    toolbox.register("evaluate", eval_func) #앞서 만들었던 eval_func() 함수
    #교배 연산자 등록
    toolbox.register("mate", tools.cxTwoPoint) 
    #변이 연산자 등록
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    
    #자식을 생성할 개체를 선택하는 연산자, 이름도 직관적으로 selTournament다
    toolbox.register("select", tools.selTournament, tournsize=3)

    return toolbox

In [5]:
#비트 수 정의하기
num_bits = 75 
#앞서 지정한 매개변수로 툴박스 생성하기
toolbox = create_toolbox(num_bits)

#난수 발생의 시드 지정하기
random.seed(7)

#500개의 개체로 구성된 초기 집단 생성하기
population = toolbox.population(n=500)
#교배와 변이 확률 정의하기
probab_crossing, probab_mutating = 0.5, 0.2
#알고리즘이 끝날 때까지 반복할 세대 수 정의하기
num_generations = 60

In [8]:
#적합도 함수를 이용해 집단을 구성하는 모든 개체를 평가한다.
print("\nStarting the evolution process")
#집단 전체 평가하기
fitness = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitness) : 
    ind.fitness.values = fit

#지정한 세대 수만큼 작업을 반복
print("\nEvaluated", len(population), "individuals") 
#세대 수만큼 반복
for g in range(num_generations) : 
    print("\n===== Generation", g)
    
    #다음 세대의 개체 선택하기 
    offspring = toolbox.select(population, len(population))
    #선택한 개체 복제
    offspring = list(map(toolbox.clone, offspring))

    #자식에 대해 교배와 변이 
    for child1, child2 in zip(offspring[::2], offspring[1::2]) : 
        #두 개체 교배하기
        if random.random() < probab_crossing : 
            toolbox.mate(child1, child2) 

            #자식에 대한 적합도 "지우기"
            del child1.fitness.values
            del child2.fitness.values
        
    #변이하기
    for mutant in offspring : 
        #개체 변이하기
        if random.random() < probab_mutating : 
            toolbox.mutate(mutant) 
            del mutant.fitness.values 

    #유효하지 않은 적합도로 개체 평가하기
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses) : 
        ind.fitness.values = fit
    print("Evaluated", len(invalid_ind), "individuals")

    #전체 집단을 자식 개체로 바꾸기
    population[:] = offspring

    #전체 적합도를 하나의 리스트에 모으고 통계값 출력하기
    fits = [ind.fitness.values[0] for ind in population]

    length = len(population) 
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits) 
    std = abs(sum2/length - mean**2)**0.5

    print("Min =", min(fits), ", Max =", max(fits))
    print("Average =", round(mean, 2), ", Standard deviation =", round(std, 2))
print("\n==== End of evolution")


Starting the evolution process

Evaluated 500 individuals

===== Generation 0
Evaluated 292 individuals
Min = 60.0 , Max = 75.0
Average = 70.77 , Standard deviation = 2.8

===== Generation 1
Evaluated 307 individuals
Min = 63.0 , Max = 75.0
Average = 72.53 , Standard deviation = 2.0

===== Generation 2
Evaluated 301 individuals
Min = 66.0 , Max = 75.0
Average = 73.41 , Standard deviation = 1.49

===== Generation 3
Evaluated 279 individuals
Min = 68.0 , Max = 75.0
Average = 73.63 , Standard deviation = 1.46

===== Generation 4
Evaluated 306 individuals
Min = 67.0 , Max = 75.0
Average = 73.65 , Standard deviation = 1.53

===== Generation 5
Evaluated 312 individuals
Min = 66.0 , Max = 75.0
Average = 73.66 , Standard deviation = 1.63

===== Generation 6
Evaluated 308 individuals
Min = 66.0 , Max = 75.0
Average = 73.74 , Standard deviation = 1.48

===== Generation 7
Evaluated 314 individuals
Min = 69.0 , Max = 75.0
Average = 73.86 , Standard deviation = 1.34

===== Generation 8
Evaluated 3

In [9]:
best_ind = tools.selBest(population, 1)[0]
print("\nBest individual:\n", best_ind)
print("\nNumber of ones:", sum(best_ind))


Best individual:
 [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0]

Number of ones: 45
